# Copyright Netherlands eScience Center <br>
** Function     : Packing the subdaily sea ice concentration fields into weekly fields from ERA-Interim as netCDF** <br>
** Author       : Yang Liu ** <br>
** First Built  : 2019.05.17 ** <br>
** Last Update  : 2019.05.17 ** <br>
Description     : This notebook aims to pack the subdaily SIC fields to weekly SIC fields from ERA-Interim.<br>
Return Values   : netCDF4 <br>
Caveat          : The data is subdaily mean of the spatial distribution of sea ice from 40N to 90N, dated from 1979 to 2017. It has sampling times as:<br>
0:00 + 6:00 + 12:00 + 18:00 <br>
For the calculation of weekly fields, we assume each month consist of 4 weeks. The first 3 weeks including 7 days. The 4th week contain the rest of the days in that month.

In [1]:
import numpy as np
import scipy as sp
import time as tttt
from netCDF4 import Dataset,num2date
import os

In [2]:
################################   Input zone  #########################################
# specify data path
#datapath = '/home/ESLT0068/WorkFlow/Core_Database_DeepLearn/ECMWFens'
datapath = '/home/ESLT0068/WorkFlow/Core_Database_DeepLearn/S2S/NCEPens'
########################################################################################

In [3]:
#datapath_full = os.path.join(datapath,"sic_weekly_S2S_ensemble_2015_2017.nc")
datapath_full = os.path.join(datapath,"sic_weekly_S2S_NCEP_ensemble_2015_2017.nc")
#datapath_full = os.path.join(datapath,"sic_weekly_S2S_NCEP_control_2015_2017.nc")
# get the variable keys
var_key = Dataset(datapath_full)

In [4]:
sic = var_key.variables['sic'][:]
lat = var_key.variables['latitude'][:]

In [15]:
################################   control  #########################################
print(sic.shape)
sic_1 = sic[1,-1,3,7:20,0:24]
np.set_printoptions(precision=2)
print(sic_1)

(3, 48, 6, 31, 240)
[[ 6.26e-01  4.83e-01  3.57e-01  2.71e-01  1.99e-01  1.26e-01  6.38e-02
   3.18e-02  1.58e-02  4.10e-02  2.08e-01  2.82e-01  5.60e-01 -1.00e+00
  -1.00e+00  8.87e-01 -1.00e+00  8.10e-01  8.28e-01  8.24e-01  8.77e-01
   9.19e-01  9.20e-01  9.19e-01]
 [ 1.25e-01  3.62e-02  6.49e-03  1.50e-03  0.00e+00  0.00e+00  0.00e+00
   2.00e-04  1.31e-03  3.48e-02  4.15e-02 -1.00e+00 -1.00e+00  6.31e-01
   6.65e-01  8.98e-01  8.72e-01  7.96e-01  8.18e-01  8.10e-01  6.12e-01
   4.52e-01  4.99e-01  7.14e-01]
 [ 8.44e-03  1.20e-03  1.43e-05  0.00e+00  0.00e+00  0.00e+00  0.00e+00
   0.00e+00  3.29e-04  5.89e-03  4.76e-03 -1.00e+00  3.74e-03  1.59e-02
   7.14e-05  2.63e-02  5.85e-02  9.02e-02  1.41e-01  6.09e-02  9.51e-03
   1.75e-02  4.41e-02  5.63e-02]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
   0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
   0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
   0.00e+00  0.00

In [6]:
################################   ensemble  #########################################
print(sic.shape)
sic_1 = sic[0,0,-1,3,7:20,0:24]
print(sic_1)
print("----------------------------")
sic_2 = sic[12,0,-1,3,7:20,0:24]
print(sic_2)
print("----------------------------")
print(sic_1 == sic_2)

(15, 1, 48, 6, 31, 240)
[[ 6.45685714e-01  4.89314286e-01  3.62857143e-01  2.78957143e-01
   2.03914286e-01  1.29828571e-01  6.78285714e-02  3.94714286e-02
   2.07714286e-02  7.76428571e-02  3.51057143e-01  4.62842857e-01
   6.46514286e-01 -1.00000000e+00 -1.00000000e+00  8.42014286e-01
  -1.00000000e+00  7.83385714e-01  8.11071429e-01  8.29528571e-01
   8.90357143e-01  9.15500000e-01  9.18442857e-01  9.15371429e-01]
 [ 1.31271429e-01  3.95857143e-02  7.61428571e-03  1.75714286e-03
   4.28571429e-05  0.00000000e+00  1.00000000e-04  6.57142857e-04
   3.50000000e-03  5.36285714e-02  7.81857143e-02 -1.00000000e+00
  -1.00000000e+00  6.75785714e-01  6.97357143e-01  8.85285714e-01
   8.43671429e-01  8.00857143e-01  8.28114286e-01  8.30085714e-01
   6.79657143e-01  5.80785714e-01  6.21957143e-01  7.73471429e-01]
 [ 9.54285714e-03  1.40000000e-03  4.28571429e-05  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.42857143e-05
   9.14285714e-04  8.65714286e-03  5.28571429e-03 

In [ ]:
# save output datasets
# we only pack our timeseries from 1979 to 2016
def create_netcdf_point (SIC, period, week, latitude, longitude, output_path):
    print ('*******************************************************************')
    print ('*********************** create netcdf file*************************')
    print ('*******************************************************************')
    print("Start creating netcdf file for SIV from 1979 to 2017.")
    # wrap the datasets into netcdf file
    # 'NETCDF3_CLASSIC', 'NETCDF3_64BIT', 'NETCDF4_CLASSIC', and 'NETCDF4'
    data_wrap = Dataset(output_path + os.sep + 'sic_weekly_erai_1979_2017.nc','w',format = 'NETCDF4')
    # create dimensions for netcdf data
    year_wrap_dim = data_wrap.createDimension('year', len(period))
    week_wrap_dim = data_wrap.createDimension('week', len(week))
    lat_wrap_dim = data_wrap.createDimension('latitude', len(latitude))
    lon_wrap_dim = data_wrap.createDimension('longitude', len(longitude))
    # create coordinate variables for 3-dimensions
    year_wrap_var = data_wrap.createVariable('year',np.int32,('year',))
    week_wrap_var = data_wrap.createVariable('week',np.int32,('week',))
    lat_wrap_var = data_wrap.createVariable('latitude',np.float32,('latitude',))
    lon_wrap_var = data_wrap.createVariable('longitude',np.float32,('longitude',))    
    # create the actual 4-d variable
    SIC_wrap_var = data_wrap.createVariable('sic',np.float64,('year','week','latitude','longitude'))

    # global attributes
    data_wrap.description = 'Weekly Sea Ice Concentration'
    # variable attributes
    lat_wrap_var.units = 'degree_north'
    lon_wrap_var.units = 'degree_east'
    SIC_wrap_var.units = 'fraction of sea ice'
    SIC_wrap_var.long_name = 'sea ice concentration'

    # writing data
    year_wrap_var[:] = period
    week_wrap_var[:] = week
    lat_wrap_var[:] = latitude
    lon_wrap_var[:] = longitude
    SIC_wrap_var[:] = SIC

    # close the file
    data_wrap.close()
    print ("Create netcdf file successfully")

In [ ]:
if __name__=="__main__":
    ####################################################################
    ######  Create time namelist matrix for variable extraction  #######
    ####################################################################
    # date and time arrangement
    # namelist of month and days for file manipulation
    namelist_month = ['01','02','03','04','05','06','07','08','09','10','11','12']
    # index of months
    period = np.arange(start_year,end_year+1,1)
    index_month = np.arange(1,13,1)
    index_week = np.arange(1,49,1)
    ####################################################################
    ######       Extract invariant and calculate constants       #######
    ####################################################################
    # get invariant from benchmark file
    Dim_year = len(period)
    Dim_month = len(index_month)
    Dim_week = len(index_week)
    Dim_latitude = 67
    Dim_longitude = 480
    #############################################
    #####   Create space for stroing data   #####
    #############################################
    # data pool
    pool_sic = np.zeros((Dim_year,Dim_week,Dim_latitude,Dim_longitude),dtype = float)
    # loop for calculation
    for i in period:
        for j in index_month:
            var_key = var_key_retrieve(datapath,i,j)
            # get the key of each variable
            latitude = var_key.variables['latitude'][:]
            longitude = var_key.variables['longitude'][:]
            sic_weekly = retriver(var_key,latitude,longitude)
            pool_sic[i-1979,j*4-4:j*4,:,:] = sic_weekly
    ####################################################################
    ######                 Data Wrapping (NetCDF)                #######
    ####################################################################
    create_netcdf_point(pool_sic, period, index_week,
                        latitude, longitude, output_path)
    print ('Packing 2D fields of ERA-Interim on surface level is complete!!!')
    print ('The output is in sleep, safe and sound!!!')